In [1]:
# some_file.py
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../src')

In [2]:
import importlib as imt 
import etl
import utils
import dataset
import clustering_kproto
imt.reload(utils)
imt.reload(etl)
imt.reload(dataset)
imt.reload(clustering_kproto)

<module 'clustering_kproto' from '../src/clustering_kproto.py'>

In [3]:
from clustering_kproto import plot_costs
from clustering_kproto import Cluster_Regression
from clustering_kproto import test_cluster_prediction
from clustering_kproto import predict_one

In [4]:
import pandas as pd
import numpy as np

### prepare data

In [5]:
data = dataset.Data_2019()
data.load()
data.clean()

/home/jaejun/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [6]:
data.df.groupby(by="new_flag_name")["property_code"].count()

new_flag_name
Crowne Plaza Hotels and Resorts     25
Doubletree by Hilton                15
Embassy Suites                      18
Hampton Inn                         42
Hampton Inn and Suites              36
Hilton                              10
Hilton Garden Inn                  152
Holiday                              8
Home2                               23
Homewood Suites                     38
Others                              23
Tru by Hilton                       76
Name: property_code, dtype: int64

In [13]:
# clustering by brand
# data.df['cluster'] = data.df['new_flag_name'].astype('category')
# data.df['cluster'].cat.codes

In [8]:
#data.df.info()

### prepare features

In [7]:
features = ['hotel_size', 'new_flag_name', 'region', 'location_type']
X = data.df[features].to_numpy()
y = data.df["revenue"].to_numpy()

### prepare test and train set

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
#shuffle
X, y = shuffle(X, y, random_state = 77)

#split for holdout data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

### scale down numerical feature, num_of_rooms

In [10]:
X_norm = X.copy()

In [22]:
X_norm[0]

array(['big', 'Embassy Suites', 'M', 'Interstate'], dtype=object)

    Parameters
    -----------
    n_clusters : int, optional, default: 8
        The number of clusters to form as well as the number of
        centroids to generate.
    max_iter : int, default: 100
        Maximum number of iterations of the k-modes algorithm for a
        single run.
    num_dissim : func, default: euclidian_dissim
        Dissimilarity function used by the algorithm for numerical variables.
        Defaults to the Euclidian dissimilarity function.
    cat_dissim : func, default: matching_dissim
        Dissimilarity function used by the kmodes algorithm for categorical variables.
        Defaults to the matching dissimilarity function.
    n_init : int, default: 10
        Number of time the k-modes algorithm will be run with different
        centroid seeds. The final results will be the best output of
        n_init consecutive runs in terms of cost.
    init : {'Huang', 'Cao', 'random' or a list of ndarrays}, default: 'Cao'
        Method for initialization:
        'Huang': Method in Huang [1997, 1998]
        'Cao': Method in Cao et al. [2009]
        'random': choose 'n_clusters' observations (rows) at random from
        data for the initial centroids.
        If a list of ndarrays is passed, it should be of length 2, with
        shapes (n_clusters, n_features) for numerical and categorical
        data respectively. These are the initial centroids.
    gamma : float, default: None
        Weighing factor that determines relative importance of numerical vs.
        categorical attributes (see discussion in Huang [1997]). By default,
        automatically calculated from data.
    verbose : integer, optional
        Verbosity mode.
    random_state : int, RandomState instance or None, optional, default: None
        If int, random_state is the seed used by the random number generator;
        If RandomState instance, random_state is the random number generator;
        If None, the random number generator is the RandomState instance used
        by `np.random`.
    n_jobs : int, default: 1
        The number of jobs to use for the computation. This works by computing
        each of the n_init runs in parallel.
        If -1 all CPUs are used. If 1 is given, no parallel computing code is
        used at all, which is useful for debugging. For n_jobs below -1,
        (n_cpus + 1 + n_jobs) are used. Thus for n_jobs = -2, all CPUs but one
        are used.
        
        matching_dissim
        jaccard_dissim_binary
        jaccard_dissim_label
        euclidean_dissim
        ng_dissim

### clustering

In [23]:
from kmodes.kmodes import KModes
from kmodes.util.dissim import jaccard_dissim_label
from kmodes.util.dissim import matching_dissim
from kmodes.util.dissim import euclidean_dissim

kmode =  KModes(n_clusters = 13, init = 'Huang', max_iter = 100,
                     cat_dissim=jaccard_dissim_label, 
                     n_init =10, verbose = 1, random_state=77, n_jobs=1) 
clusters = kmode.fit_predict(X_norm)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 167, cost: 138.14999999999986
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 112, cost: 131.06666666666663
Run 2, iteration: 2/100, moves: 13, cost: 131.06666666666663
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 191, cost: 123.6833333333334
Run 3, iteration: 2/100, moves: 67, cost: 118.93333333333337
Run 3, iteration: 3/100, moves: 5, cost: 117.8833333333334
Run 3, iteration: 4/100, moves: 4, cost: 117.8833333333334
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 122, cost: 133.1166666666665
Run 4, iteration: 2/100, moves: 26, cost: 130.59999999999988
Run 4, iteration: 3/100, moves: 17, cost: 130.59999999999988
Init: initializing centroids
Init: initializing clusters
Starting iterat

### Attributes

cluster_centroids_ : array, [n_clusters, n_features]
    Categories of cluster centroids
labels_ :
    Labels of each point
cost_ : float
    Clustering cost, defined as the sum distance of all points to
    their respective cluster centroids.
n_iter_ : int
    The number of iterations the algorithm ran for.
epoch_costs_ :
    The cost of the algorithm at each epoch from start to completion.
gamma : float
    The (potentially calculated) weighing factor.

In [24]:
centroid = kmode.cluster_centroids_
cost = kmode.epoch_costs_ 

In [17]:
cost

[131.19999999999996, 128.64999999999995, 129.68333333333325]

In [27]:
centroid

array([['medium', 'Hilton Garden Inn', 'N', 'Interstate'],
       ['big', 'Hilton Garden Inn', 'W', 'Urban'],
       ['small', 'Hampton Inn', 'S', 'Suburban'],
       ['medium', 'Home2', 'S', 'Urban'],
       ['small', 'Tru by Hilton', 'S', 'Suburban'],
       ['medium', 'Hilton Garden Inn', 'S', 'Interstate'],
       ['medium', 'Hilton Garden Inn', 'S', 'Suburban'],
       ['big', 'Doubletree by Hilton', 'M', 'Urban'],
       ['medium', 'Hilton Garden Inn', 'W', 'Urban'],
       ['small', 'Tru by Hilton', 'S', 'Interstate'],
       ['medium', 'Tru by Hilton', 'S', 'Suburban'],
       ['big', 'Embassy Suites', 'S', 'Urban'],
       ['small', 'Hampton Inn', 'M', 'Interstate']], dtype='<U20')

### prepare prediction datasets

In [28]:
df = data.df.copy()
df['cluster'] = clusters
X_new = df["cluster"].to_numpy()
y_new = data.target_spor().to_numpy()

In [29]:
test_cluster_prediction(X_new, y_new)

Train Data MSE: 0.55
Train Data R2: 0.05
Holdout Data MSE: 0.43
Holdout Data R2: -0.02


In [31]:


x = ['big', 'Crowne Plaza Hotels and Resorts', 'Airport', utils.region('IL')]
y_hat = predict_one(kmode, x)

In [32]:
y_hat

1

### List of Cluster

In [33]:
#number of properties for each cluster
df.groupby(by="cluster")['property_code'].count() 

cluster
0     44
1     40
2     48
3     34
4     34
5     54
6     55
7     36
8     15
9     35
10    15
11    45
12    11
Name: property_code, dtype: int64

In [34]:
from tabulate import tabulate
lst_columns = ['property_name', 'flag_name', 'num_of_rooms', 'city', 'state', 'location_type', 'spor', 'profit_margin']
for i in range(df["cluster"].max()+1):
    print(tabulate(df[df['cluster'] == i][lst_columns].head(5), headers='keys', tablefmt='psql'))
    #df[df['cluster'] == i][lst_columns].head(5)

+----+--------------------------------------------+---------------------------------+----------------+------------+---------+-----------------+-----------+-----------------+
|    | property_name                              | flag_name                       |   num_of_rooms | city       | state   | location_type   |      spor |   profit_margin |
|----+--------------------------------------------+---------------------------------+----------------+------------+---------+-----------------+-----------+-----------------|
|  0 | Aston Waikiki Beach Hotel                  | Aston                           |            625 | Honolulu   | HI      | Urban           | 0.0856803 |        0.53522  |
| 17 | Crowne Plaza Charlotte Executive Park      | Crowne Plaza Hotels and Resorts |            300 | Charlotte  | NC      | Suburban        | 1.7364    |        0.625514 |
| 22 | Crowne Plaza Greenville-I-385-Roper Mtn Rd | Crowne Plaza Hotels and Resorts |            202 | Greenville | SC      | Urba

In [32]:
pd.options.display.max_rows = 100

In [37]:
df[df['cluster'] == 3][lst_columns].sort_values(by='flag_name')

,property_name,flag_name,num_of_rooms,city,state,location_type,spor,profit_margin
1,Avid Hotel Meridian,Avid,87,Oklahoma City,OK,Airport,0.564489,0.510323
41,DoubleTree Suites by Hilton Hotel Boston - Cam...,Doubletree by Hilton,308,Boston,MA,Urban,4.632502,0.708547
50,Doubletree by Hilton Hotel San Diego - Del Mar,Doubletree by Hilton,224,San Diego,CA,Interstate,0.879263,0.652463
109,Hampton Inn & Suites Trophy Club,Hampton Inn,94,Trophy Club,TX,Suburban,1.014780,0.680809
116,Hampton Inn Chicago West Loop,Hampton Inn,210,Chicago,IL,Urban,0.509387,0.640560
126,Hampton Inn Jupiter/Juno Beach,Hampton Inn,89,Juno Beach,FL,Suburban,1.807850,0.513333
220,Hilton Garden Inn Jackson Pearl,Hampton Inn,90,Pearl,MS,Interstate,1.207653,0.661644
82,Hampton Inn & Suites Columbia South,Hampton Inn and Suites,124,Columbia,MD,Suburban,1.846941,0.579426
88,Hampton Inn & Suites Hood River,Hampton Inn and Suites,88,Hood River,OR,Interstate,2.705626,0.549547
104,Hampton Inn & Suites Saskatoon Airport,Hampton Inn and Suites,137,Saskatoon,SK,Urban,0.279177,0.485520


In [34]:
df.groupby(by="flag_name")["property_code"].count()

flag_name
Aston                                1
Avid                                 3
Best Western Plus                    1
Candlewood Suites                    5
Comfort Suites                       2
Crowne Plaza Hotels and Resorts     25
Curio by Hilton                      3
Doubletree by Hilton                15
Embassy Suites                      18
Hampton Inn                         42
Hampton Inn and Suites              36
Hilton                               6
Hilton Garden Inn                  152
Hilton Suites                        1
Holiday Inn                          1
Holiday Inn Express                  4
Holiday Inn Express & Suites         3
Home2                               23
Homewood Suites                     38
Hotel Indigo                         1
Independent                          1
Intercontinental Hotels Group        1
La Quinta Inns and Suites            1
Liv Dev                              2
Quality Suites                       1
Red Lion Hotel 

In [ ]:
# Save models if are satisfied with clusters 

In [24]:
import pickle

with open('../web/data/kproto.pkl', 'wb') as handle:
    pickle.dump(kproto, handle, protocol=pickle.HIGHEST_PROTOCOL)
df.to_pickle('../web/data/model.pkl')